In [1]:
import numpy as np
import random
import math
from pprint import pprint

# Importing QISKit
from qiskit import QuantumCircuit, QuantumProgram
import Qconfig

# Import basic plotting tools
from qiskit.tools.visualization import plot_histogram

# Quantum program setup 
Q_program = QuantumProgram()
Q_program.set_api(Qconfig.APItoken, Qconfig.config['url']) # set the APIToken and API url

In [2]:
## Creating registers

qr = Q_program.create_quantum_register("qr", 2)
cr = Q_program.create_classical_register("cr", 2)

In [3]:
## Creating a shared singlet state

singlet = Q_program.create_circuit("singlet", [qr], [cr])
singlet.x(qr[0])
singlet.x(qr[1])
singlet.h(qr[0])
singlet.cx(qr[0],qr[1])

In [4]:
## Alice's and Bob's measurement choice strings

aliceMeasurementsChoices = []
bobMeasurementsChoices = []

measuremensChoicesLength = 50 # length of the strings

# random strings generation
for i in range(measuremensChoicesLength):
    aliceMeasurementsChoices.append(random.randint(0, 2))
    bobMeasurementsChoices.append(random.randint(0, 2))

In [5]:
#debug# Print measurement choices strings
print(aliceMeasurementsChoices)
print(bobMeasurementsChoices)

In [6]:
## Alice's measurement circuits

# measurement of spin projection onto (1; 0; 0) vector; standard X-measurement
measureA1 = Q_program.create_circuit('measureA1', [qr], [cr])
measureA1.h(qr[0])
measureA1.measure(qr[0],cr[0])

# measurement of spin projection onto (1/sqt(2); 0; 1/(sqrt(2)) vector
# projection onto 1/sqrt(2)(Z+X) eigenstates
measureA2 = Q_program.create_circuit('measureA2', [qr], [cr])
measureA2.s(qr[0])
measureA2.h(qr[0])
measureA2.t(qr[0])
measureA2.h(qr[0])
measureA2.measure(qr[0],cr[0])

# measurement of spin projection onto (0; 0; 1) vector; standard Z-measurement
measureA3 = Q_program.create_circuit('measureA3', [qr], [cr])
measureA3.measure(qr[0],cr[0])

## Bob's measurement circuits

# measurement of spin projection onto (1/sqt(2); 0; 1/(sqrt(2)) vector
# projection onto 1/sqrt(2)(Z+X) eigenstates
measureB1 = Q_program.create_circuit('measureB1', [qr], [cr])
measureB1.s(qr[1])
measureB1.h(qr[1])
measureB1.t(qr[1])
measureB1.h(qr[1])
measureB1.measure(qr[1],cr[1])

# measurement of spin projection onto (0; 0; 1) vector; standard Z-measurement
measureB2 = Q_program.create_circuit('measureB2', [qr], [cr])
measureB2.measure(qr[1],cr[1])

# measurement of spin projection onto (-1/sqt(2); 0; 1/(sqrt(2)) vector
# projection onto 1/sqrt(2)(Z-X) eigenstates
measureB3 = Q_program.create_circuit('measureB3', [qr], [cr])
measureB3.s(qr[1])
measureB3.h(qr[1])
measureB3.tdg(qr[1])
measureB3.h(qr[1])
measureB3.measure(qr[1],cr[1])

In [7]:
## Arrays of Alice's and Bob's measurement circuits

aliceMeasurements = [measureA1, measureA2, measureA3]
bobMeasurements = [measureB1, measureB2, measureB3]

In [8]:
circuits = [] # prepared circuits

for k in range(measuremensChoicesLength):
    circuitName = 'A' + str(aliceMeasurementsChoices[k]+1) + 'B' + str(bobMeasurementsChoices[k]+1)
    Q_program.add_circuit(circuitName, singlet + aliceMeasurements[aliceMeasurementsChoices[k]] + bobMeasurements[bobMeasurementsChoices[k]]) # Measurement of Alice's and Bob's qubits
    circuits.append(circuitName) # add measurement of singlet to circuits array

In [9]:
## Device status

pprint(Q_program.get_backend_status('ibmqx2'))

{'available': False, 'backend': 'ibmqx2', 'busy': False, 'pending_jobs': 3026}


In [10]:
## Execute circuits

# backend that is used for the calculation
backend = 'local_qasm_simulator' # simulator
#backend = 'ibmqx2' # real device

result = Q_program.execute(circuits, backend=backend, shots=1, max_credits=3, wait=10, timeout=240, silent=False)

running on backend: local_qasm_simulator


In [11]:
#debug# Print basis choices and posterior states

for k in range(measuremensChoicesLength):
    print(str(circuits[k]) + ': ' + str(result.get_counts(circuits[k])))

In [12]:
## Writing measurement results

aliceResults = [] # Alice's results
bobResults = [] # Bob's results

for k in range(measuremensChoicesLength):
    res = result.get_counts(circuits[k]) # type = dictionary; in this case it has one key (state) and one value
    if '00' in res: # check if the key is 00 (if posterior state is |00>)
        aliceResults.append(-1) # Alice got the result -1 
        bobResults.append(-1) # Bob got the result -1 
    if '01' in res: # check if the key is 01 (if posterior state is |01>)
        aliceResults.append(-1) # Alice got the result -1 
        bobResults.append(1) # Bob got the result +1 
    if '10' in res:
        aliceResults.append(1)
        bobResults.append(-1)
    if '11' in res:
        aliceResults.append(1)
        bobResults.append(1)

In [13]:
#debug# Print measurement results

print(aliceResults)
print(bobResults)

[1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1]
[-1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, 1]


In [14]:
## Basis choises check stage
#
# Alice and Bob reveal their strings with basis choices
#
# If in k-th measurement A and B used the same basis,
# then they write the result of the k-th measurement as the bit of the sifted key 

aliceSiftedKey = [] # Alice's siffted key string
bobSiftedKey = [] # Bob's siffted key string

# the sifted key consist of the results obtained by projecting onto the same eigenstates
# that is A2+B1 and A3+B2 measurements
for k in range(measuremensChoicesLength):
    if (aliceMeasurementsChoices[k] == 1 and bobMeasurementsChoices[k] == 0) or (aliceMeasurementsChoices[k] == 2 and bobMeasurementsChoices[k] == 1):
        aliceSiftedKey.append(aliceResults[k]) # write Alice's k-th result as a key bit
        bobSiftedKey.append(-bobResults[k]) # write inversed Bob's k-th result as a key bit   

In [15]:
#debug# Print sifted keys; have to be the same

print(aliceSiftedKey)
print(bobSiftedKey)

[-1, 1, 1, -1, 1, -1, 1, 1, 1]
[-1, 1, 1, -1, 1, -1, 1, 1, 1]


In [16]:
## CHSH inequality test
# inefficient; could be performed in the previous block

countA1B1 = [0, 0, 0, 0]
countA1B3 = [0, 0, 0, 0]
countA3B1 = [0, 0, 0, 0]
countA3B3 = [0, 0, 0, 0]
c11 = 0
c13 = 0
c31 = 0
c33 = 0
norm = 0

for k in range(measuremensChoicesLength):
    
    res = result.get_counts(circuits[k])
    
    if (aliceMeasurementsChoices[k] == 0 and bobMeasurementsChoices[k] == 0):
        if '00' in res:
            countA1B1[0] += 1
        if '01' in res:
            countA1B1[1] += 1
        if '10' in res:
            countA1B1[2] += 1
        if '11' in res:
            countA1B1[3] += 1
        norm += 1
        c11 += 1
    if (aliceMeasurementsChoices[k] == 0 and bobMeasurementsChoices[k] == 2):
        if '00' in res:
            countA1B3[0] += 1
        if '01' in res:
            countA1B3[1] += 1
        if '10' in res:
            countA1B3[2] += 1
        if '11' in res:
            countA1B3[3] += 1
        norm += 1
        c13 += 1        
    if (aliceMeasurementsChoices[k] == 2 and bobMeasurementsChoices[k] == 0):
        if '00' in res:
            countA3B1[0] += 1
        if '01' in res:
            countA3B1[1] += 1
        if '10' in res:
            countA3B1[2] += 1
        if '11' in res:
            countA3B1[3] += 1
        norm += 1
        c31 += 1        
    if (aliceMeasurementsChoices[k] == 2 and bobMeasurementsChoices[k] == 2):
        if '00' in res:
            countA3B3[0] += 1
        if '01' in res:
            countA3B3[1] += 1
        if '10' in res:
            countA3B3[2] += 1
        if '11' in res:
            countA3B3[3] += 1
        norm += 1
        c33 += 1
